Library


In [26]:
import pickle
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

Path


In [27]:
PATH_MAIN = r"C:\Users\Lulay\Documents\GitHub\Dasar-Kecerdasan-Artificial_Tugas-Besar"
# PATH_MAIN = r"D:\Kuliah\Matkul\Semester 4\DASAR KECERDASAN ARTIFICIAL (DKA)\[2] Tugas\[3] Tugas Besar\Dasar-Kecerdasan-Artificial_Tugas-Besar"

PATH_DATASET = f"{PATH_MAIN}/Dataset"

Konstanta


In [28]:
SAMPLE_RATE = 48000
BATCH_SIZE = 32
SEGMENT_DURATION = int(1 * SAMPLE_RATE)
OVERLAP_DURATION = int(0.5 * SAMPLE_RATE)
RANDOM_STATE = 21
BUFFER_SIZE = 1000

Random Seed


In [29]:
RANDOM_SEED = 21

try:
    from sklearn.utils import check_random_state
    random_state = check_random_state(RANDOM_SEED)
except ImportError:
    pass

# Dataset


Load Dataset Raw


In [30]:
with open(f"{PATH_DATASET}/dataset_fe.pkl", "rb") as f:
    dataset = pickle.load(f)

Checking


In [31]:
len(dataset)

1894

In [32]:
dataset[0]

{'path': 'C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar\\Dataset\\xeno-canto\\19655.mp3',
 'name': '19655.mp3',
 'timestamp': [0.0, 1.0],
 'label': 2,
 'raw': array([-6.6490856e-13, -2.0503420e-13,  1.9726230e-12, ...,
         1.7010156e-02,  1.5655354e-02,  1.3218733e-02], dtype=float32),
 'rms': 0.011059444,
 'spectral_flatness': 0.011423797,
 'mfcc': array([[-9.2709552e+02, -9.2709552e+02, -8.6692578e+02, ...,
         -4.9831238e+02, -4.7504291e+02, -3.9764847e+02],
        [ 0.0000000e+00,  0.0000000e+00, -1.4798780e+01, ...,
         -8.1901199e+01, -5.0254303e+01,  2.3999417e+00],
        [ 0.0000000e+00,  0.0000000e+00, -5.1890976e+01, ...,
         -2.6562943e+02, -2.3930214e+02, -1.4375620e+02],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  8.7950077e+00, ...,
          7.3784947e-01,  1.3385715e+00,  2.1685948e+00],
        [ 0.0000000e+00,  0.0000000e+00, -1.2696293e+01, ...,
          5.2022276e+00,  3.3751645e+00, -6.7336679e+00],


`Label 1 untuk suara non-burung dan 0 untuk suara burung`


Assign variabel X dan y


In [33]:
X = [data["mfcc"] for data in dataset]
y = [int(data["label"]) for data in dataset]

Check


In [34]:
pd.Series(y).value_counts()

1    685
0    652
2    557
Name: count, dtype: int64

In [35]:
print("Data:\n", X[0])
print("Label: ", y[0])

Data:
 [[-9.2709552e+02 -9.2709552e+02 -8.6692578e+02 ... -4.9831238e+02
  -4.7504291e+02 -3.9764847e+02]
 [ 0.0000000e+00  0.0000000e+00 -1.4798780e+01 ... -8.1901199e+01
  -5.0254303e+01  2.3999417e+00]
 [ 0.0000000e+00  0.0000000e+00 -5.1890976e+01 ... -2.6562943e+02
  -2.3930214e+02 -1.4375620e+02]
 ...
 [ 0.0000000e+00  0.0000000e+00  8.7950077e+00 ...  7.3784947e-01
   1.3385715e+00  2.1685948e+00]
 [ 0.0000000e+00  0.0000000e+00 -1.2696293e+01 ...  5.2022276e+00
   3.3751645e+00 -6.7336679e+00]
 [ 0.0000000e+00  0.0000000e+00  6.4206371e+00 ...  2.9003916e+00
  -1.4942620e+00  4.1397813e-01]]
Label:  2


Squeeze


In [36]:
X = np.array(X)
X = X.reshape(X.shape[0], -1)

Split


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE, stratify=y_test)

Normalisasi


In [38]:
scl = StandardScaler()
X_train_scaled = scl.fit_transform(X_train)
X_test_scaled = scl.transform(X_test)
X_val_scaled = scl.transform(X_val)

Save scaler


In [39]:
joblib.dump(scl, f"{PATH_MAIN}/Model/a_scaler.pkl")

['C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar/Model/a_scaler.pkl']

Check


In [40]:
display(
    X_train_scaled.shape,
    X_test_scaled.shape,
    X_val_scaled.shape
)

(1515, 1880)

(189, 1880)

(190, 1880)

# Train


Model yang akan digunakan untuk klasifikasi suara burung:

1. LGBM Classifier
2. MLP Classifier


## LGBM


Inisialisasi


In [41]:
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)

training


In [42]:
lgbm_model.fit(X_train_scaled, y_train)

LGBMClassifier(random_state=21, verbose=-1)

Evaluasi


In [43]:
y_pred = lgbm_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred, average="macro")}")
print(f"Recall: {recall_score(y_val, y_pred, average="macro")}")
print(f"F1 Score: {f1_score(y_val, y_pred, average="macro")}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

Validasi
Accuracy: 0.8052631578947368
Precision: 0.8073552425665103
Recall: 0.8058849363197189
F1 Score: 0.8065270677218294
Confusion Matrix:
 [[52 10  4]
 [ 8 56  5]
 [ 5  5 45]]


In [48]:
y_pred = lgbm_model.predict(X_test_scaled)

print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred, average="macro")}")
print(f"Recall: {recall_score(y_test, y_pred, average="macro")}")
print(f"F1 Score: {f1_score(y_test, y_pred, average="macro")}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Test
Accuracy: 0.8783068783068783
Precision: 0.8782234420475946
Recall: 0.8807907778496015
F1 Score: 0.8792193397689677
Confusion Matrix:
 [[55  5  5]
 [ 7 59  2]
 [ 2  2 52]]


Save


In [49]:
joblib.dump(lgbm_model, f'{PATH_MAIN}/model/a_lgbm.pkl')

['C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar/model/a_lgbm.pkl']

## MLP


Inisialisasi


In [50]:
mlp_model = MLPClassifier(random_state=RANDOM_STATE, verbose=False)

training


In [51]:
mlp_model.fit(X_train_scaled, y_train)

MLPClassifier(random_state=21)

Evaluasi


In [52]:
y_pred = mlp_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred, average="macro")}")
print(f"Recall: {recall_score(y_val, y_pred, average="macro")}")
print(f"F1 Score: {f1_score(y_val, y_pred, average="macro")}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

Validasi
Accuracy: 0.7842105263157895
Precision: 0.786678481760449
Recall: 0.7846728151075978
F1 Score: 0.7849567755079567
Confusion Matrix:
 [[49 11  6]
 [ 8 56  5]
 [ 4  7 44]]


In [53]:
y_pred = mlp_model.predict(X_test_scaled)

print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred, average="macro")}")
print(f"Recall: {recall_score(y_test, y_pred, average="macro")}")
print(f"F1 Score: {f1_score(y_test, y_pred, average="macro")}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Test
Accuracy: 0.8359788359788359
Precision: 0.8359256349829799
Recall: 0.8366946778711485
F1 Score: 0.8362722604610511
Confusion Matrix:
 [[52  6  7]
 [ 8 58  2]
 [ 5  3 48]]


Save


In [54]:
joblib.dump(mlp_model, f'{PATH_MAIN}/model/a_mlp.pkl')

['C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar/model/a_mlp.pkl']